<a href="https://colab.research.google.com/github/sanjivanipatil123/100DaysOfMLLearning/blob/main/Football_Data_Test_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
import pandas as pd

# Load the dataset from an Excel file
file_path = r"/content/Football Data Test Task2.xlsx"  # Raw string to handle backslashes
data = pd.read_excel(file_path)

# Convert the 'Date' column to a proper datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Print the first few rows to verify
print(data.head())


   Incremental_ID Div       Date Time       HomeTeam    AwayTeam  FTHG  FTAG  \
0               1  E0 2005-08-13  NaN    Aston Villa      Bolton     2     2   
1               2  E0 2005-08-13  NaN        Everton  Man United     0     2   
2               3  E0 2005-08-13  NaN         Fulham  Birmingham     0     0   
3               4  E0 2005-08-13  NaN       Man City   West Brom     0     0   
4               5  E0 2005-08-13  NaN  Middlesbrough   Liverpool     0     0   

  FTR  HTHG  ...  HST AST  HF  AF  HC  AC  HY  AY  HR  AR  
0   D     2  ...    2   6  14  16   7   8   0   2   0   0  
1   A     0  ...    5   5  15  14   8   6   3   1   0   0  
2   D     0  ...    7   4  12  13   6   6   1   2   0   0  
3   D     0  ...    8   3  13  11   3   6   2   3   0   0  
4   D     0  ...    2   7  17  11   5   0   2   3   1   0  

[5 rows x 25 columns]


In [4]:
# List of blue columns to process
blue_columns = ['FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']

In [5]:
# Function to compute aggregated metrics
def compute_last_n_matches(df, team_col, stat_col, n, team, match_type):
    # Filter matches involving the team
    team_matches = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)].copy()
    team_matches['is_home'] = team_matches['HomeTeam'] == team

In [7]:
def compute_last_n_matches(df, team_col, stat_col, n, team, match_type):
    # Filter matches involving the team
    team_matches = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)].copy()
    team_matches['is_home'] = team_matches['HomeTeam'] == team
    # Add relevant stats
    team_matches[stat_col + '_adjusted'] = team_matches.apply(
        lambda row: row[stat_col] if row['is_home'] else 0, axis=1
    ) if match_type == 'Home' else team_matches.apply(
        lambda row: 0 if row['is_home'] else row[stat_col], axis=1
    )

In [9]:
# Function to compute aggregated metrics
def compute_last_n_matches(df, team_col, stat_col, n, team, match_type):
    # Filter matches involving the team
    team_matches = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)].copy()
    team_matches['is_home'] = team_matches['HomeTeam'] == team
    # Add relevant stats
    team_matches[stat_col + '_adjusted'] = team_matches.apply(
        lambda row: row[stat_col] if row['is_home'] else 0, axis=1
    ) if match_type == 'Home' else team_matches.apply(
        lambda row: 0 if row['is_home'] else row[stat_col], axis=1
    )
    # Compute rolling sum
    rolling_sum = team_matches[stat_col + '_adjusted'].rolling(n).sum().iloc[-1] # This line was indented incorrectly
    return rolling_sum

In [14]:
# Manipulate data for all teams and blue columns
teams = data['HomeTeam'].unique()  # Use 'data' instead of 'df'
manipulated_data = []

for team in teams:
    team_data = {'Team': team}
    for col in blue_columns:
        for n in [5, 15, 38]:
            team_data[f'{col}_L{n}_Home'] = compute_last_n_matches(data, 'HomeTeam', col, n, team, 'Home') # Use 'data' instead of 'df'
            team_data[f'{col}_L{n}_Away'] = compute_last_n_matches(data, 'AwayTeam', col, n, team, 'Away') # Use 'data' instead of 'df'
            manipulated_data.append(team_data) # Append team_data to manipulated_data after each iteration

In [15]:
# Convert to DataFrame
manipulated_df = pd.DataFrame(manipulated_data)

In [16]:
# Save the manipulated data
manipulated_df.to_csv("manipulated_data.csv", index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')